<a href="https://colab.research.google.com/github/jonrtaylor/twitch/blob/master/neutralized_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### THIS NOTEBOOK WILL NOT RUN IN GOOGLE COLAB ###

import pandas as pd
import numpy as np
import scipy

tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
features = [c for c in tournament_data if c.startswith("feature")]


from sklearn.preprocessing import MinMaxScaler
def _neutralize(df, preds, feats, proportion=1.0):
    scores = df[preds]
    exposures = df[feats].values
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    return scores / scores.std()
def _normalize(df):
    X = (df.rank(method="first") - 0.5) / len(df)
    return scipy.stats.norm.ppf(X)
def normalize_and_neutralize(df, preds, feats, proportion=1.0):
    # Convert the scores to a normal distribution
    df[preds] = _normalize(df[preds])
    df[preds] = _neutralize(df, preds, feats, proportion)
    return df[preds]

predfile = pd.read_csv('example_predictions_twitch.csv') #import your prediction file in this line

tournament_data['preds'] = predfile.prediction_kazutsugi
tournament_data['neutralized_preds'] = tournament_data.groupby("era").apply(lambda x: normalize_and_neutralize(x, ["preds"], features, 1)) #set your proportion as something between 1 (fully neutralized) and something >0)
scaler = MinMaxScaler()
tournament_data['neutralized_preds'] = scaler.fit_transform(tournament_data[['neutralized_preds']])

In [ ]:
tournament_submission = tournament_data[['id', 'neutralized_preds']].copy()
tournament_submission.columns = ['id', 'prediction']
tournament_submission.to_csv('twitch_example_fully_neutralized.csv', index=False) #save the new prediction file in this line